Вариант 6:
$$Q(\lambda) = \int\limits_0^\infty f(t) e^{-\lambda t}\,dt$$
$$Q(\lambda) \approx \sum\limits_{k = 0}^{N} \frac{1}{\lambda^{k + 1}} f^{(k)}(0) + O\left(\frac{1}{\lambda^{N + 1}}\right),\ \lambda \to \infty$$

$$f(t) = \sqrt{1 + t^2}$$
а) Численное интегрирование

Для начала заметим, что:
$$\sqrt{1 + t^2} < 1 + t,\ \ \ t > 0$$
Это нам поможет оценить "хвост" следующего интеграла:
$$
\begin{align*}
\int\limits_0^\infty \sqrt{1 + t^2} e^{-\lambda t}\, dt = \int\limits_0^A \sqrt{1 + t^2} e^{-\lambda t}\, dt + \int\limits_A^\infty \sqrt{1 + t^2} e^{-\lambda t}\, dt,
\end{align*}
$$
где первый интеграл можно легко посчитать численно. Оценим второй:
$$\int\limits_A^\infty \sqrt{1 + t^2} e^{-\lambda t}\, dt < \int\limits_A^\infty (1 + t) e^{-\lambda t}\, dt = \int\limits_A^\infty e^{-\lambda t}\, dt + \int\limits_A^\infty t e^{-\lambda t}\, dt = \frac{2 + A}{\lambda} e^{-\lambda A}$$

С помощью полученной оценки можно эффективно определить такое A, чтобы можно было пренебречь значением "хвоста".

б) Асимптотический ряд

$f(t)$ - четная, значит коэффициенты в ряде Маклорена при нечетных степенях равны 0, то есть производные нечетного порядка равны 0. С нечетными степенями сложнее, потому позволю себе воспользоваться готовым разложением в ряд Маклорена. Можно заметить, что производные четных порядков можно получить следующим образом.
Пусть $a_n = f^{(2n)}(0)$. Легко можно посчитать и получить, что $a_0 = a_1 = 1$. Тогда последующие производные можно вычислить рекурсивно:
$$a_n = (-1)^{n + 1} (2n - 3)(2n - 1)a_{n - 1} = (-1)^{n + 1}(4n^2 - 8n + 3)a_{n - 1},\ \ n = 2,\, 3,\, ...$$

Тогда наш ряд:
$$Q_n(\lambda) = \sum\limits_{k = 0}^{n} \frac{f^{(k)}(0)}{\lambda^{k + 1}}$$

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# правая разностная схема
def right_diff(func, x0, h):
    return (func(x0 + h) - func(x0)) / h

# формула Симпсона
def integral_simpson(func, interval, n = 10):
    (a, b) = interval
    h = (b - a) / n
    ans = func(a) + func(b) + 4 * func(a + h / 2)
    for i in range(1, n):
        ans += 2 * func(a + i * h) + 4 * func(a + (i + 1 / 2) * h)
    ans *= h / 6
    return ans

# функция вычисления эффективного конечного верхнего предела
def eff_lim(A, lm, err = 10**-5, max_iter = 10):
    iter = 0
    while iter < max_iter and (2 + A) / lm * np.exp(-lm * A) > err:
        A *= 2
        iter += 1
    if iter < max_iter:
        return A
    return None

# асимптотический ряд в нуле
def asymptotic_series_at_0(lm, n):
    sum = (1 / lm if n >= 0 else 0) + (1 / (lm ** 3) if n >= 2 else 0)
    last_der = 1
    for i in range(4, n + 1, 2):
        k = int(i / 2)
        last_der *= (-1) ** (k + 1) * (4 * k ** 2 -8 * k + 3)
        sum += last_der / lm ** (i + 1)
    return sum

In [3]:
lms = [2, 3, 4, 5, 10, 20]
ns = [2, 3, 4, 6, 8, 10, 20]
f = lambda t: np.sqrt(1 + t ** 2)
for_laplace = lambda f, lm: lambda t: f(t) * np.exp(-lm * t)

for lm in lms:
    errors = []
    
    A = eff_lim(1, lm, 10 ** -15)
    if A > 0:
        # численный интеграл с шагом 1 / 1000
        Q = integral_simpson(for_laplace(f, lm), (0, A), 1000 * A)
        print(f"Q({lm})\t=", Q)
    else:
        print("Well... Something went wrong for lambda =", lm)

    Q_n = []
    for n in ns:
        Q_n.append(asymptotic_series_at_0(lm, n))
        errors.append(abs(Q - Q_n[-1]))
        print(f"Q{n}({lm})\t=", Q_n[-1])
    print("Наименьшая погрешность асимтотического ряда достигается при n =", ns[errors.index(min(errors))]) 
    print()

Q(2)	= 0.5920301195022017
Q2(2)	= 0.625
Q3(2)	= 0.625
Q4(2)	= 0.53125
Q6(2)	= 0.1796875
Q8(2)	= 3.255859375
Q10(2)	= 51.70556640625
Q20(2)	= -10622928880.69255
Наименьшая погрешность асимтотического ряда достигается при n = 2

Q(3)	= 0.3641289901130674
Q2(3)	= 0.37037037037037035
Q3(3)	= 0.37037037037037035
Q4(3)	= 0.35802469135802467
Q6(3)	= 0.33744855967078186
Q8(3)	= 0.417466849565615
Q10(3)	= 0.9775948788294467
Q20(3)	= -2094750.2033609252
Наименьшая погрешность асимтотического ряда достигается при n = 4

Q(4)	= 0.26381561644459206
Q2(4)	= 0.265625
Q3(4)	= 0.265625
Q4(4)	= 0.2626953125
Q6(4)	= 0.25994873046875
Q8(4)	= 0.2659568786621094
Q10(4)	= 0.2896139621734619
Q20(4)	= -4861.018481192689
Наименьшая погрешность асимтотического ряда достигается при n = 4

Q(5)	= 0.20732903093746036
Q2(5)	= 0.20800000000000002
Q3(5)	= 0.20800000000000002
Q4(5)	= 0.20704000000000003
Q6(5)	= 0.20646400000000004
Q8(5)	= 0.20727040000000005
Q10(5)	= 0.20930252800000004
Q20(5)	= -43.1386602574984
Наиме